In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from keras.layers import Lambda, Input, GaussianNoise,concatenate, Dense, Dropout, Conv2D, Add, UpSampling2D, Dot, Conv2DTranspose, Activation, Reshape, InputSpec, LeakyReLU, Flatten, BatchNormalization
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.utils.generic_utils import Progbar
from time import time

from keras import backend as K
from keras.engine import *
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils.generic_utils import func_dump
from keras.utils.generic_utils import func_load
from keras.utils.generic_utils import deserialize_keras_object
from keras.utils.generic_utils import has_arg
from keras.utils import conv_utils
from keras.layers import Dense, Conv1D, Conv2D, Conv3D, Conv2DTranspose, Embedding
import tensorflow as tf
import cv2
import os
from imutils import paths
from keras.models import load_model
import tensorflow_probability as tfp
from keras.applications.vgg19 import VGG19
from tensorflow_addons.layers import SpectralNormalization
from keras.activations import swish
from keras.initializers import glorot_uniform, glorot_normal, GlorotUniform
import pandas as pd 
from keras.constraints import max_norm, UnitNorm


In [6]:
from platform import python_version
print(python_version())

import tensorflow as tf
print(tf.version.VERSION)

print(tf.__version__)

3.9.10
2.6.2
2.6.2


In [ ]:
#Hyperperemeter

do_augmentations = 0
BATCHSIZE=4 # 64
LEARNING_RATE = 0.0002 # initial 0.0002
TRAINING_RATIO = 1
BETA_1 = 0.0
BETA_2 = 0.9
EPOCHS = 500
BN_MIMENTUM = 0.1
BN_EPSILON  = 0.00002 
latent_dim = 350
img_size = 64
channels = 1
img_shape = (img_size, img_size, channels)
nmax = 200
ker_init = GlorotUniform(seed=None)
noise_var = 1
load_model_true = 1
do_augmentations = 0
upload_saved_model = 0
data_path = '/home/odedrot' # user input

In [ ]:
import GPUtil
from tabulate import tabulate

print("="*40, "GPU Details", "="*40)

gpus = GPUtil.getGPUs()
list_gpus = []

for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"

    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free mem", "used mem", "total mem",
                                   "temperature")))

In [ ]:
# load data
margin = 15
def load_images(directory):
    imagePaths = list(paths.list_images(directory))
    data = []
    images_name = []
    i = 1
    for imagePath in imagePaths:
        # images_name.append(imagePath.split("/")[5])
        image = cv2.imread(imagePath) # load the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # swap color channels
        image = image[margin:img_size-margin , margin:img_size-margin] 
        image = cv2.resize(image, (img_size,img_size) , interpolation = cv2.INTER_AREA)
        data.append(image)
        i = i+1
        if i>nmax:
            break
    req_images = np.array(data)
    req_images =  np.array(req_images).astype("float32")
    req_images = np.expand_dims(req_images, axis=-1)
    return req_images# , images_name


################################################################################################
# train
images0 = load_images(data_path + '/DISCOVERY-master/GENDER/IMAGES/TRAIN_0') 
images1 = load_images(data_path + '/DISCOVERY-master/GENDER/IMAGES/TRAIN_1') 
X_train = np.concatenate((images0, images1))

################################################################################################
# test
images0 = load_images(data_path + '/DISCOVERY-master/GENDER/IMAGES/TEST_0') 
images1 = load_images(data_path + '/DISCOVERY-master/GENDER/IMAGES/TEST_1') 

X_test = np.concatenate((images0, images1))


print('X_train' , X_train.shape)
print('X_test' , X_test.shape)
##################################################################################################

In [ ]:
@tf.function
def brightness(x):
    x = tf.image.random_brightness(x, 0.15)
    x = tf.clip_by_value(x, 0, 1)
    return x
@tf.function
def contrast(x):
    x = tf.image.random_contrast(x, 0.85, 1.15)
    x = tf.clip_by_value(x, 0, 1)
    return x
@tf.function
def rotate(x):
    orient = np.random.choice([1,2,3], 1)[0] # 90,180,270
    x = tf.image.rot90(x, k=orient)
    # x = tf.contrib.image.rotate(y, angles=2.356194490192345, interpolation='NEAREST')
    return x
@tf.function
def flip_ver(x):
    x = tf.image.random_flip_up_down(x)
    return x
@tf.function
def flip_hor(x):
    x = tf.image.random_flip_left_right(x)
    return x
@tf.function
def add_noise(x):
    x_noise = tf.random.normal(shape=(img_size, img_size, 1), mean=0.0, stddev=0.03, dtype=tf.float32)
    x = tf.add(x, x_noise) 
    x = tf.clip_by_value(x, 0, 1)
    return x
@tf.function
def resize_crop(x):
    rand_size = tf.random.uniform(shape=[],  minval=int(0.75 * img_size),  maxval=1 * img_size, dtype=tf.int32,)
    crop = tf.image.random_crop(x, (rand_size, rand_size, x.shape[-1]))
    x = tf.image.resize(crop, (img_size, img_size))
    return x
@tf.function
def blur(x):
    # s = np.random.random()
    # x = tfa.image.gaussian_filter2d(image=x, sigma=s)     
    x = tfa.image.gaussian_filter2d(x, filter_shape=(3,3), sigma=0.3)
    return x

def augment_img(img):
    img_aug = np.copy(img)
    ### flip hor
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = flip_hor(img_aug)
    ### flip ver
    # rand_val = np.random.uniform(0,1)
    # if rand_val > 0.5:
    #     img_aug = flip_ver(img_aug)
    ### rotate
    # rand_val = np.random.uniform(0,1)
    # if rand_val > 0.5:
    #     img_aug = rotate(img_aug)
    ### resize crop
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = resize_crop(img_aug)
        
    ### blur
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = blur(img_aug)
    ### brightness
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = brightness(img_aug)
    ### contrast
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = contrast(img_aug)
    ### noise
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        img_aug = add_noise(img_aug)
    return img_aug


def augment_imgs(imgs):
    imgs_aug = []
    for i in range(len(imgs)):
        img_aug = augment_img(imgs[i])
        imgs_aug.append( img_aug )
    imgs_aug = np.array(imgs_aug)
    return imgs_aug

In [ ]:
def sample_images(imgs):
    r, c = 2, 10
    fig, axs = plt.subplots(r, c, figsize=(20,4))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.show()

In [ ]:
class DenseSN(Dense):
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                                 initializer=initializers.RandomNormal(0, 1),
                                 name='sn',
                                 trainable=False)
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True
        
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                 W_bar = K.reshape(W_bar, W_shape)  
        output = K.dot(inputs, W_bar)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output
    

class ConvSN2D(Conv2D):

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        #Spectral Normalization
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
                
        outputs = K.conv2d(
                inputs,
                W_bar,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs    

    
class ConvSN2DTranspose(Conv2DTranspose):

    def build(self, input_shape):
        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (self.filters, input_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=4, axes={channel_axis: input_dim})
        self.built = True  
    
    def call(self, inputs):
        input_shape = K.shape(inputs)
        batch_size = input_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = input_shape[h_axis], input_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides
        if self.output_padding is None:
            out_pad_h = out_pad_w = None
        else:
            out_pad_h, out_pad_w = self.output_padding

        # Infer the dynamic output shape:
        out_height = conv_utils.deconv_length(height,
                                              stride_h, kernel_h,
                                              self.padding,
                                              out_pad_h)
        out_width = conv_utils.deconv_length(width,
                                             stride_w, kernel_w,
                                             self.padding,
                                             out_pad_w)
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)
            
        #Spectral Normalization    
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
        self.kernel = W_bar
        
        outputs = K.conv2d_transpose(
            inputs,
            self.kernel,
            output_shape,
            self.strides,
            padding=self.padding,
            data_format=self.data_format)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs
     

In [ ]:
GaussianNoise_std = 0.003

def res_block_down(layer_input, filters):
    d = BatchNormalization()(layer_input)
    d = Activation('relu')(d)
    d = ConvSN2D(filters, kernel_size=3, strides=2, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = ConvSN2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d_res = ConvSN2D(filters, kernel_size=1, strides=2, padding='same')(layer_input)
    d = Add()([d, d_res])
    return d

def res_block_up(layer_input, filters):
    d = BatchNormalization()(layer_input)
    d = Activation('relu')(d)
    d = UpSampling2D(size=(2, 2), interpolation="nearest")(d)
    d = ConvSN2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = ConvSN2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d_res = UpSampling2D(size=(2, 2), interpolation="nearest")(layer_input)
    d_res = ConvSN2D(filters, kernel_size=1, strides=1, padding='same')(d_res)
    d = Add()([d, d_res])
    return d

# @tf.keras.utils.register_keras_serializable(package='Custom', name='Wvar_reg')
def Wvar_reg(weight_matrix):
    return tf.math.reduce_mean(tf.math.reduce_variance(weight_matrix))

def BuildEncoder():
    enc_input = Input(shape=img_shape, name='encoder_input') 
    X = ConvSN2D(64, kernel_size=3, padding='same', activation='relu')(enc_input)
    X = res_block_down(X, 128)
    X = res_block_down(X, 256)
    X = res_block_down(X, 512)
    X = res_block_down(X, 512)
    
    X = Flatten()(X)
    X = GaussianNoise(stddev=GaussianNoise_std)(X)
    X = swish(X) #############
        
    z = DenseSN(latent_dim)(X)
    z = GaussianNoise(stddev=GaussianNoise_std)(z)
    z = BatchNormalization()(z)
    
    z_split = Lambda( lambda x: tf.split(x,num_or_size_splits=latent_dim//5,axis=1))(z) 
    # clfscore = Dense(1, activation='sigmoid')(z_split[0])
    clfscore = Dense(1, activation='sigmoid', kernel_regularizer=Wvar_reg)(z_split[0])
    
    encoder = Model(enc_input, [z, clfscore], name='encoder')
    
    return encoder

def BuildGenerator():
    dec_input = Input(shape=(latent_dim, ), name='decoder_input')
    last_conv_shape = (4, 4, 512)
    X = DenseSN(last_conv_shape[0] * last_conv_shape[1] * last_conv_shape[2])(dec_input)
    X = GaussianNoise(stddev=GaussianNoise_std)(X)
    X = Reshape((last_conv_shape[0], last_conv_shape[1], last_conv_shape[2]))(X)
    
    X = res_block_up(X, 512)
    X = res_block_up(X, 512)
    X = res_block_up(X, 256)
    X = res_block_up(X, 128)
    
    X = ConvSN2D(1, (3, 3), strides = (1,1), padding = 'same', kernel_initializer = ker_init)(X)
    X = Activation('sigmoid')(X)
    decoder = Model(dec_input, X, name='decoder')
    return decoder

In [ ]:
def res_block_up(layer_input, filters):
    d = BatchNormalization()(layer_input)
    d = Activation('relu')(d)
    d = UpSampling2D(size=(2, 2), interpolation="nearest")(d)
    d = ConvSN2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = ConvSN2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d_res = UpSampling2D(size=(2, 2), interpolation="nearest")(layer_input)
    d_res = ConvSN2D(filters, kernel_size=1, strides=1, padding='same')(d_res)
    d = Add()([d, d_res])
    return d

def BuildGenerator():
    dec_input = Input(shape=(latent_dim, ), name='decoder_input')
    last_conv_shape = (4, 4, 512)
    X = DenseSN(last_conv_shape[0] * last_conv_shape[1] * last_conv_shape[2])(dec_input)
    X = GaussianNoise(stddev=GaussianNoise_std)(X)
    X = Reshape((last_conv_shape[0], last_conv_shape[1], last_conv_shape[2]))(X)
    
    X = res_block_up(X, 512)
    X = res_block_up(X, 512)
    X = res_block_up(X, 256)
    X = res_block_up(X, 128)
    
    X = ConvSN2D(1, (3, 3), strides = (1,1), padding = 'same', kernel_initializer = ker_init)(X)
    X = Activation('sigmoid')(X)
    decoder = Model(dec_input, X, name='decoder')
    return decoder

In [ ]:
def BuildDiscriminator1D():
    model = Sequential()
    model.add(Dense(2048, input_dim=latent_dim))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(alpha=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(alpha=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(alpha=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(alpha=0.2))
    model.add((Dense(2048)))
    model.add(LeakyReLU(alpha=0.2))
    model.add((Dense(2048)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)
    return Model(encoded_repr, validity, name='discriminator')

def build_disentangleNET():
    disentangleNET_input = Input(shape=img_shape, name='disentangleNET_input') # batchx128x128x1    
    x = Conv2D(32, 5, strides=2, padding="same", name="rec_conv1_preact")(disentangleNET_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(64, 5, strides=2, padding="same", name="rec_conv2_preact")(x) 
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, 5, strides=2, padding="same", name="rec_conv3_preact")(x)  
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, 5, strides=2, padding="same", name="rec_conv4_preact")(x)  
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Flatten()(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    disentangleNET_output = Dense(latent_dim, activation='softmax')(x)
    disentangleNET_value = Dense(1)(x)
    disentangleNET = Model(disentangleNET_input, [disentangleNET_output], name="disentangleNET")
    return disentangleNET

In [ ]:
def gray2rgb(tensor):
    tensor = tf.image.grayscale_to_rgb(tensor)
    return tensor

def polynomial_kernel(features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

def dist_sim_wrapper(model):
    def dist_sim_loss(y_true, y_pred):
        real_features = model( Lambda(gray2rgb)(y_true) ) 
        generated_features = model( Lambda(gray2rgb)(y_pred) )  
        
        loss = K.mean(real_features - generated_features) 
        
        # kernel_real = polynomial_kernel(real_features, real_features)
        # kernel_generated = polynomial_kernel( generated_features, generated_features)
        # kernel_cross = polynomial_kernel(real_features, generated_features)
        # batch_size = tf.shape(real_features)[0]
        # batch_size_f = tf.cast(batch_size, dtype=tf.float32)
        # mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (batch_size_f * (batch_size_f - 1.0))
        # mean_kernel_generated = tf.reduce_sum(kernel_generated * (1.0 - tf.eye(batch_size))) / (batch_size_f * (batch_size_f - 1.0))
        # mean_kernel_cross = tf.reduce_mean(kernel_cross)
        # loss = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross
        
        return loss
    return dist_sim_loss

# def calc_cov_metric(z):
#     cov = tfp.stats.covariance(z)
#     diag_mean = tf.reduce_sum(tf.linalg.diag_part(cov)) / cov.shape[0]
#     off_diag_mean = ( tf.reduce_sum(cov) - tf.reduce_sum(tf.linalg.diag_part(cov)) ) / (cov.shape[0]*2 - cov.shape[0])
#     cov_loss = tf.reduce_mean((diag_mean - noise_var)**2) + tf.reduce_mean(off_diag_mean**2)
#     return cov_loss, diag_mean, off_diag_mean

def z_mean_var_loss(z):
    loss = K.abs(K.var(z)-1) + K.abs(K.mean(z))
    return K.mean(loss)

def get_off_diag_values(x):
    x_flat = tf.reshape(x,[-1])[:-1]
    x_reshape = tf.reshape(x_flat,[x.shape[0]-1, x.shape[0]+1])[:, 1:]
    off_diag_values = tf.reshape(x_reshape,[-1])
    return off_diag_values
def cov_loss_terms(z_batch):
    z_batch = z_batch - tf.reduce_mean(z_batch, axis=0)
    z_std = tf.sqrt(tf.math.reduce_variance(z_batch, axis=0) + 0.0001)
    z_std_loss = tf.reduce_mean( tf.nn.relu(1 - z_std) )
    cov = (tf.transpose(z_batch) @ z_batch) / (z_batch.shape[0] ) # cov = 1/N*(z.T*z)  tfp.stats.covariance(z_batch)
    diag_cov = tf.linalg.diag_part(cov) # vector of vars (z_std**2)  positives
    diag_cov_mean = tf.reduce_mean(tf.abs(diag_cov))    
    off_diag_loss    = tf.reduce_mean(tf.abs( get_off_diag_values(cov) ))
    off_diag_mean_10 = tf.reduce_mean(tf.abs(get_off_diag_values(cov[0:10,0:10])))
    return cov, z_std_loss, diag_cov_mean , off_diag_loss, off_diag_mean_10 


def calc_dimcor_loss(z, scores):
    cor = []
    N_dim = 5
    for i in range(N_dim):
        cor.append( tfp.stats.correlation(z[:,i] , scores[:,0], sample_axis=0, event_axis=None) )
    first_cor_loss = 1 - tf.reduce_mean(tf.abs(cor)) # with or without abs?
    
    for i in range(N_dim+1, latent_dim):
        cor.append( tfp.stats.correlation(z[:,i] , scores[:,0], sample_axis=0, event_axis=None) )
    rest_cor_loss = tf.reduce_mean(tf.abs(cor)) # with or without abs?
    
    return first_cor_loss + rest_cor_loss


def Wmae_loss(y_true, y_pred):
    mae = K.mean(K.abs(y_pred - y_true), axis=-1)
    th = 0.05
    booli = K.cast(K.greater(mae, th), 'float32') # keep bad samples (high loss)
#     booli = K.cast(K.less_equal(mae, th), 'float32') # keep good samples (small loss)
    wmae = booli * mae
    wmae = K.sum(wmae) / K.sum(booli) 
    return wmae

# def model_perc_loss_wrapper(model):
#     def model_perc_loss(y_true, y_pred):
#         # weights = [1,1,1]
#         img_outputs = model( y_true ) 
#         recon_img_outputs = model( y_pred ) 
#         layers_losses = []
#         for i in range(len(recon_img_outputs)):
#             mae_losses = K.abs(img_outputs[i] - recon_img_outputs[i])
#             layers_losses.append(K.mean(mae_losses))
#         avg_loss = K.mean(tf.convert_to_tensor(layers_losses))
#         return avg_loss
#     return model_perc_loss

def perc_loss(model, y_true, y_pred):
    # weights = [1,1,1]
    y_true = Lambda(gray2rgb)(y_true)
    y_pred = Lambda(gray2rgb)(y_pred)
    img_outputs = model( y_true ) 
    recon_img_outputs = model( y_pred ) 
    layers_losses = []
    for i in range(len(recon_img_outputs)):
        mae_losses = K.abs(img_outputs[i] - recon_img_outputs[i])
        layers_losses.append(K.mean(mae_losses))
    avg_loss = K.mean(tf.convert_to_tensor(layers_losses))
    return avg_loss

def perturbate_embedding(z_image_batch):
    GT_onehot_vecs = np.zeros((z_image_batch.shape[0], latent_dim)) # vectors of zeros which will get 1 in a random dim
    pert_onehot_vecs = np.copy(GT_onehot_vecs) # vectors of zeros which will get a value in a random dim , it is added to z in the graph
    for i in range(z_image_batch.shape[0]):
        pert_dim = np.random.randint(latent_dim, size=1)[0] # choose a dim
        values = np.linspace(-6, 6, num=100)
        values[np.where( (values<-1) | (values>1) )]
        pert_value = tf.random.shuffle(values)[0] # choose a value from values
        pert_onehot_vecs[i][pert_dim] = pert_value # change z to selected value
        GT_onehot_vecs[i][pert_dim] = 1 # put value 1 in one hot
    pert_onehot_vecs = np.array(pert_onehot_vecs)
    GT_onehot_vecs = np.array(GT_onehot_vecs)
    return pert_onehot_vecs, GT_onehot_vecs

def polynomial_kernel(features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0
def z_mmd_loss2(real_features, generated_features):
    kernel_real = polynomial_kernel(real_features, real_features)
    kernel_generated = polynomial_kernel( generated_features, generated_features)
    kernel_cross = polynomial_kernel(real_features, generated_features)
    batch_size = tf.shape(real_features)[0]
    batch_size_f = tf.cast(batch_size, dtype=tf.float32)
    mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (batch_size_f * (batch_size_f - 1.0))
    mean_kernel_generated = tf.reduce_sum(kernel_generated * (1.0 - tf.eye(batch_size))) / (batch_size_f * (batch_size_f - 1.0))
    mean_kernel_cross = tf.reduce_mean(kernel_cross)
    kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross
    return kid

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.001)
cce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)


def multi_layer_second_directional_derivative(G, z, x, epsilon):
    G_z = [G(z)]
    G_to_x = [G(z + x)]
    G_from_x = [G(z - x)]
    sdd = [(G2x - 2 * G_z_base + Gfx) / epsilon**2 for G2x, G_z_base, Gfx in zip(G_to_x, G_z, G_from_x)] # len=1 , sdd[0] 64,64,64,1
    return sdd
def hessian_penalty(G, z, k):
    epsilon=0.1
    second_orders = []
    for _ in range(k): 
        rademacher_tensors = tfp.random.rademacher(tf.shape(z)) # (bs, zdim)
        x = epsilon * rademacher_tensors # (bs, zdim)
        central_second_order = multi_layer_second_directional_derivative(G, z, x, epsilon)   ########## # len=1 , sdd[0] 64,64,64,1
        second_orders.append(central_second_order) # (k,1,  64,64,64,1)
    
    sum_of_penalties = 0 
    for activ_n in zip(*second_orders): 
        second_orders__ = tf.stack(activ_n) # (100, 64, 64, 64, 1)
        _, var_tensor = tf.nn.moments(second_orders__, axes=(0,)) # var_tensor (64, 64, 64, 1)
        penalty = tf.reduce_max(var_tensor)  # (1,) (scalar) 
        sum_of_penalties += penalty 
    return sum_of_penalties



In [ ]:

CNN_saved_name = 'classifier.h5' # new
CNN_clf_model_path = data_path + 'GENDER/GENDER_SAVED_MODELS/' + CNN_saved_name
CNN_clf_model = load_model(CNN_clf_model_path , compile=True)
CNN_clf_model.trainable = False # we dont train the CNN

clf_outputs = [
               CNN_clf_model.get_layer('block2_conv1').output,
               CNN_clf_model.get_layer('block2_conv2').output,
               CNN_clf_model.get_layer('block3_conv2').output,
               CNN_clf_model.get_layer('block3_conv3').output,
               CNN_clf_model.get_layer('block5_conv2').output,
               CNN_clf_model.get_layer('block5_conv3').output,
               CNN_clf_model.get_layer('flatten').output,
               CNN_clf_model.get_layer('dense_1').output,]
                  
CNN_clf_Model_outputs = Model(inputs=CNN_clf_model.input, outputs = clf_outputs)
CNN_clf_Model_outputs.trainable = False # we dont train the CNN

VGG_model = VGG19(weights="imagenet", include_top=False, input_shape=(img_size, img_size,3)) # ResNet50 , VGG16, VGG19
VGG_outputs = [
               # VGG_model.get_layer('block1_conv1').output,   
               # VGG_model.get_layer('block1_conv2').output,
               # VGG_model.get_layer('block2_conv1').output,
               # VGG_model.get_layer('block2_conv2').output,   
               VGG_model.get_layer('block3_conv1').output,   
               VGG_model.get_layer('block3_conv2').output,
               VGG_model.get_layer('block3_conv3').output,
               VGG_model.get_layer('block4_conv1').output,
               VGG_model.get_layer('block4_conv2').output,
               VGG_model.get_layer('block4_conv3').output,
               VGG_model.get_layer('block4_conv4').output,
               VGG_model.get_layer('block5_conv1').output,
               VGG_model.get_layer('block5_conv2').output,
               VGG_model.get_layer('block5_conv3').output,
               VGG_model.get_layer('block5_conv4').output,]
VGG_Model_outputs = Model(inputs=VGG_model.input, outputs = VGG_outputs)
VGG_Model_outputs.trainable = False # we dont train the CNN

In [ ]:
encoder = BuildEncoder()
generator = BuildGenerator()
disentangleNET = build_disentangleNET()
discriminator1D = BuildDiscriminator1D()
# discriminatorDW = BuildDiscriminatorDW()

# load weights to model
if upload_saved_model == 1:
    model_path = data_path + 'GENDER/GENDER_SAVED_MODELS/'
    encoder.load_weights(model_path + 'encoder' + '.h5')
    generator.load_weights(model_path + 'generator' + '.h5')
    disentangleNET.load_weights(model_path + 'disentangleNET' + '.h5')
    discriminator1D.load_weights(model_path + 'discriminator1D' + '.h5')

ae_optimizer=Adam(LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2)
disc1D_optimizer=Adam(LEARNING_RATE/10, beta_1=BETA_1, beta_2=BETA_2)

In [ ]:
test_batch_size = 200
test_images = X_test[np.random.randint(0, X_test.shape[0], test_batch_size)] / 255 
test_images_scores = CNN_clf_model( test_images.repeat(3, -1) )
test_noise = np.random.normal(0,noise_var, size=(test_batch_size, latent_dim))
test_real_y = 0.9*np.ones((test_batch_size, 1))
test_fake_y = 0.1*np.ones((test_batch_size, 1))

real_y = 0.9*np.ones((BATCHSIZE, 1))
fake_y = 0.1*np.ones((BATCHSIZE, 1))

for epoch in range(EPOCHS):
    print("epoch", epoch)
    iteration = 0
    for n_batch in range(len(X_train) // BATCHSIZE): 
        # training function start
        # train encoder 
        for i in range(1):
            idx = np.random.randint(0, X_train.shape[0], 2*BATCHSIZE)
            image_batch = np.copy(X_train[idx]) / 255
            if do_augmentations == 1:
                image_batch = augment_imgs(image_batch)
                
            ## choose high perc loss images
            recon_image_batch = generator.predict(encoder.predict(image_batch)[0]) 
            samples_perc_loss = []
            for i in range(2*BATCHSIZE):
                samples_perc_loss.append( perc_loss(VGG_Model_outputs, image_batch[i:i+1], recon_image_batch[i:i+1]) )
            samples_perc_loss = np.array(samples_perc_loss)
            sorted_ind = np.argsort(samples_perc_loss)[::-1]
            # sorted_samples_perc_loss = samples_perc_loss[sorted_ind]
            # sorted_samples_perc_loss = sorted_samples_perc_loss[~np.isnan(sorted_samples_perc_loss)] # remove nan
            sorted_images = image_batch[sorted_ind]
            image_batch = sorted_images[0:BATCHSIZE]
                
            imgs_clfscore = CNN_clf_model(image_batch.repeat(3, -1))
            with tf.GradientTape() as tape:
                z_imgs, z_scores = encoder(image_batch, training=True)
                recon_imgs = generator(z_imgs, training=True)
                ## disentangler part
                pert_z_vecs, GT_onehot_vecs = perturbate_embedding(z_imgs)
                pert_imgs = generator(z_imgs + pert_z_vecs, training=True) ## only predict
                pert_diff = tf.abs(pert_imgs - recon_imgs) ### abs or not
                onehot_dim_pred = disentangleNET(pert_diff, training=True) 
                pert__pert_imgs = generator(encoder(pert_imgs, training=True)[0] - pert_z_vecs, training=True )
                
                ## disciriminator1D adv
                z_discriminator1D_out = discriminator1D(z_imgs, training=True) # adversarial 
                # z_discriminatorDW_out = discriminatorDW(tf.transpose(z_imgs), training=True) # adversarial 
                ## loss
                CLF_perc_loss = perc_loss(CNN_clf_Model_outputs, image_batch, recon_imgs)
                VGG_perc_loss = perc_loss(VGG_Model_outputs, image_batch, recon_imgs)
                # MAE_loss = K.mean(K.abs(image_batch - recon_imgs))
                z_score_loss = bce(imgs_clfscore, z_scores) + bce(z_scores , imgs_clfscore) # bce(imgs_clfscore, z_scores) # calc_dimcor_loss(z_imgs, imgs_clfscore)
                # z_score_loss_cor = calc_dimcor_loss(z_imgs, imgs_clfscore)
                disent_loss = cce(GT_onehot_vecs , onehot_dim_pred)
                # pert_diff_loss = tf.reduce_mean(pert_diff) - tf.math.reduce_variance(pert_diff)
                # pert_consistency_loss = perc_loss(CNN_clf_Model_outputs, pert_imgs, pert__pert_imgs)
                # pert_CLF_perc_loss = perc_loss(CNN_clf_Model_outputs, image_batch, pert_imgs)
                # pert_VGG_perc_loss = perc_loss(VGG_Model_outputs, image_batch, pert_imgs)
                z_discriminator1D_loss = tf.cast(bce(real_y, z_discriminator1D_out), tf.float32)
                # z_discriminatorDW_loss = tf.cast(bce( 0.9*np.ones((latent_dim, 1)) , z_discriminatorDW_out), tf.float32)
                z_mean_var_loss_ = z_mean_var_loss(z_imgs)
                cov, z_std_loss, diag_cov_mean , off_diag_loss, off_diag_mean_10 = cov_loss_terms(z_imgs)
                cov_loss = 0.5*diag_cov_mean + 0.5*z_std_loss
                # H_loss = hessian_penalty(generator, z_imgs[0:8], k=10)
                # pert_recon_loss = K.mean(K.abs(recon_imgs - generator(z_imgs + 0.3*tf.random.normal(shape=(BATCHSIZE, latent_dim)), training=True) ))
                
                ############ total loss ################
                
                recon_loss = 5*CLF_perc_loss + 5*VGG_perc_loss
                latent_space_loss = 1*z_score_loss + 1*z_discriminator1D_loss + 1*cov_loss + 1*z_mean_var_loss_ # + 1*z_discriminatorDW_loss
                perturbation_loss = 1*disent_loss
                ae_loss = recon_loss + latent_space_loss + perturbation_loss
       
                
            ## ae BP (discriminator isnt trainable)
            trainable_variables = encoder.trainable_variables + generator.trainable_variables + disentangleNET.trainable_variables 
            ae_gradients = tape.gradient(ae_loss, trainable_variables)
            ae_optimizer.apply_gradients(zip(ae_gradients, trainable_variables))
            
            ## discriminator1D/DW
            idx = np.random.randint(0, X_train.shape[0], BATCHSIZE)
            image_batch = np.copy(X_train[idx]) / 255
            if do_augmentations == 1:
                image_batch = augment_imgs(image_batch)
            rand_vecs = tf.random.normal(shape=(BATCHSIZE, latent_dim))
            with tf.GradientTape(persistent=True) as tape:
                z_imgs, z_scores = encoder(image_batch, training=True)
                z_discriminator1D_out = discriminator1D(z_imgs, training=True)
                r_discriminator1D_out = discriminator1D(rand_vecs, training=True)
                # z_discriminatorDW_out = discriminatorDW(tf.transpose(z_imgs), training=True)
                # r_discriminatorDW_out = discriminatorDW(tf.transpose(rand_vecs), training=True)
                # loss
                discriminator1D_loss = 0.5*bce(real_y, r_discriminator1D_out) + 0.5*bce(fake_y, z_discriminator1D_out)
                discriminator1D_loss = tf.cast(discriminator1D_loss, tf.float32)
                # discriminatorDW_loss = 0.5*bce( 0.9*np.ones((latent_dim, 1)) , r_discriminatorDW_out) + 0.5*bce( 0.1*np.ones((latent_dim, 1)) , z_discriminatorDW_out)
                # discriminatorDW_loss = tf.cast(discriminatorDW_loss, tf.float32)
            ## BP disc1D 
            trainable_variables = discriminator1D.trainable_variables
            disc1D_gradients = tape.gradient(discriminator1D_loss, trainable_variables)
            disc1D_optimizer.apply_gradients(zip(disc1D_gradients, trainable_variables))
            ## BP discDW 
            # trainable_variables = discriminatorDW.trainable_variables
            # discDW_gradients = tape.gradient(discriminatorDW_loss, trainable_variables)
            # discDW_optimizer.apply_gradients(zip(discDW_gradients, trainable_variables))
            
# TEST ###########################################
        iteration = iteration + 1
        if iteration % 1 == 0: 
            print('iteration ' , iteration)
            test_z_images, test_z_scores  = encoder.predict(test_images)
            test_recon_images = generator.predict(test_z_images)
            test_generated_images = generator.predict(test_noise)
            test_recon_images_scores = CNN_clf_model(test_recon_images.repeat(3, -1))

            # disent
            test_pert_z_vecs, test_GT_onehot_vecs = perturbate_embedding(test_z_images)
            test_pert_imgs = generator.predict(test_z_images + test_pert_z_vecs)
            test_pert_diff = tf.abs(test_pert_imgs - test_recon_images)
            test_onehot_dim_pred = disentangleNET.predict(test_pert_diff) ### abs or not
            test_dim_pred = np.argmax(test_onehot_dim_pred, axis=1)
            test_disentangleNET_acc = len(test_dim_pred[test_dim_pred==np.argmax(test_GT_onehot_vecs, axis=1)]) / len(test_dim_pred)
            test_pert_CLF_perc_loss = perc_loss(CNN_clf_Model_outputs, test_images, test_pert_imgs).numpy()
            test_pert_VGG_perc_loss = perc_loss(VGG_Model_outputs, test_images, test_pert_imgs).numpy()
            test_pert__pert_imgs = generator.predict(encoder.predict(test_pert_imgs)[0] - test_pert_z_vecs )
            
            # test losses
            test_CLF_loss = np.round(perc_loss(CNN_clf_Model_outputs, test_images, test_recon_images).numpy(),3)
            test_VGG_loss = np.round(perc_loss(VGG_Model_outputs, test_images, test_recon_images).numpy(),3)
            test_z_scores_loss = np.round(K.mean(K.abs(test_images_scores - test_z_scores)).numpy(),3)
            test_disent_acc = np.round(test_disentangleNET_acc,3)
            test_cov, test_z_std_loss, test_diag_cov_mean , test_off_diag_loss, test_off_diag_mean_10 = cov_loss_terms(test_z_images)
            test_pert_consistency_loss = perc_loss(CNN_clf_Model_outputs, test_pert_imgs, test_pert__pert_imgs)
            test_pert_diff_loss = tf.reduce_mean(test_pert_diff)
            
            print(  '                 train   ' ,   '  test')
            print('CLF_loss    ' , CLF_perc_loss.numpy() , '   ',  test_CLF_loss)
            print('VGG_loss    ' , VGG_perc_loss.numpy() , '   ',  test_VGG_loss)

            # print('perc_CLF_loss    ' , pert_CLF_perc_loss.numpy() , '   ',  test_pert_CLF_perc_loss)
            # print('perc_VGG_loss    ' , pert_VGG_perc_loss.numpy() , '   ',  test_pert_VGG_perc_loss)
            # print('pert_consistency_loss  ' , pert_consistency_loss.numpy(), '   ', test_pert_consistency_loss.numpy())
            print('pert_diffsize_loss  ' , pert_diff_loss.numpy(), '   ', test_pert_diff_loss.numpy())
            print('z_scores_mae_loss    ' , z_score_loss.numpy(), '   ', test_z_scores_loss)
            print('disent_loss' , cce(test_GT_onehot_vecs , test_onehot_dim_pred).numpy() , test_disent_acc )
            print('disc1D rand' , bce(test_real_y, discriminator1D(test_noise, training=True)).numpy() )
            print('disc1D z' , bce(test_fake_y, discriminator1D(test_z_images, training=True)).numpy() )
            print( 'mean_diag' , tf.reduce_mean(tf.linalg.diag_part(test_cov)).numpy(), 'off_diag_loss', test_off_diag_loss.numpy(), 'test_off_diag_mean_10', test_off_diag_mean_10.numpy()  )
            
            print('')
            print('MAE_loss' ,  K.mean(K.abs(image_batch - recon_imgs)).numpy(),' ', K.mean(K.abs(test_images - test_recon_images)).numpy())
            print('Other metrics')
            print( '[z_var, z_mean]' , np.var(test_z_images) , np.mean(test_z_images) )
            print('recon_scores_mae_loss' , np.mean(np.abs(CNN_clf_model(test_recon_images.repeat(3, -1)) - test_images_scores)) ) 
            print( 'weights: ' , np.sort(np.abs(encoder.layers[-1].get_weights()[0].flatten())) )
            
            print('')
            print('real')
            sample_images(test_images)
            print('')
            print('recon')
            sample_images(test_recon_images)
            print('')

            print('gen')
            sample_images(test_generated_images)

            _ = plt.hist(discriminator1D(test_noise, training=True).numpy().flatten(), 100, range=[0, 1], alpha = 0.5, color= 'b')
            _ = plt.hist(discriminator1D(test_z_images, training=True).numpy().flatten(), 100, range=[0, 1], alpha = 0.5, color= 'r')
            handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in ["red", "blue"] ]
            labels= ["z", "rand"]
            plt.legend(handles, labels)
            plt.show()
            
            cor_vals = []
            for i in range(latent_dim):
                cor_vals.append( np.corrcoef(test_images_scores[:,0] , test_z_images[:,i])[0][1] )
            cor_vals = np.array(cor_vals)
            plt.figure(figsize=(30,10))
            plt.bar(np.arange(latent_dim), cor_vals , width=1)
#             df = pd.concat([pd.DataFrame(test_z_images) , pd.DataFrame(test_images_scores)] , axis=1).reset_index()
#             df = df.drop( ['index'] , axis=1)
#             cor = df.corr()
#             
#             plt.bar(np.array(df.columns[:-1]), cor.iloc[-1].values[:-1] , width=1) # normalized
#             plt.xlabel("latent dimension", fontsize=50)
            plt.ylabel("correlation ", fontsize=50)
            plt.xticks(fontsize=30)
            plt.yticks(fontsize=30)
            plt.ylim([-1,1])
            plt.show()
            
            print(cor_vals[15:].max())
            
    if (epoch+1) % 1 == 0:
        save_path = data_path + 'GENDER/GENDER_SAVED_MODELS/' 
        encoder.save(save_path + 'encoder.h5', include_optimizer=False)
        generator.save(save_path + 'generator.h5', include_optimizer=False)
        disentangleNET.save(save_path + 'disentangleNET.h5', include_optimizer=False)
        discriminator1D.save(save_path + 'discriminator1D.h5', include_optimizer=False)
        
        
        